In [1]:
%autosave 0

Autosave disabled


In [2]:
%pip install git+https://github.com/jvkersch/pyconcorde

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jvkersch/pyconcorde to /tmp/pip-req-build-i1c_uyal
  Running command git clone --filter=blob:none --quiet https://github.com/jvkersch/pyconcorde /tmp/pip-req-build-i1c_uyal
  Resolved https://github.com/jvkersch/pyconcorde to commit 71232a8043a3ca79a7a6da2f48e9d89d81d37163
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyconcorde: filename=pyconcorde-0.1.0-cp310-cp310-linux_x86_64.whl size=2494568 sha256=b8d9749ceb5303fb99e29956963a33513e00f1b1572422887fd5f31e43b2998c
  Stored in directory: /tmp/pip-ephem-wheel-cache-08df_52w/wheels/be/9f/eb/e4f95e06e62f057a045679e4940e536d1b251d1ab4859c6dcc
Successfully built pyconcorde


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# install tweak file
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/concorde_class.py' '/content'
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/test.py' '/content'
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/concorde_to_replace/tsp.py' '/usr/local/lib/python3.10/dist-packages/concorde'
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/concorde_to_replace/util.py' '/usr/local/lib/python3.10/dist-packages/concorde'

In [4]:

import os
import pandas as pd
import numpy as np
import re
import collections
import pickle
from ast import literal_eval
import random
import scipy
import math
from concorde_class import Concorde
import datetime
import subprocess

#Define Path of Files

Mounted at /content/drive


In [25]:
# path for the 'Data' folder provided by DP
dir_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data'

#Functions

In [ ]:
# generate workflow for file storage /  instance creation

## Volume Matrix Manipulation and Poisson Instances Generation


In [17]:
# create dataframes for a region everyday

def generate_region_volume(region_name):
  # path of folder
  dir_region_instances = dir_loc + '/Instances/' + region_name
  dir_region_volume = dir_loc + '/Volumes/'
  # dir_region_districts = dir_region_instances + '/Districts'

  # load post_object-to-route_pos_id and post_point-to-post_object files 
  po_file = pd.read_csv(dir_region_instances + '/post_order_id_mapping.dat', sep='\t', names=('PostObjectId', 'RoutePosID'))
  pp_file = pd.read_csv(dir_region_instances + '/post_point_information.dat', sep='\t', names=('PostPointId', 'PostObjectId'))

  # pp_file adjustment by splitting list of post object ids
  pp_file['PostObjectId'] = pp_file['PostObjectId'].apply(literal_eval)
  pp_file = pp_file.explode('PostObjectId', ignore_index=True)

  # complete  post object list-district mapping
  region_district_df = generate_region_district(region_name = region_name)
  region_district_df.rename(columns = {'PostObjectID' : 'PostObjectId'}, inplace = True)

  # list of volume file paths
  day_names = []
  vol_path_list = []
  vol_day_map = {}
  day_map = {'mo' : 'Monday',
             'di' : 'Tuesday',
             'mi' : 'Wednesday',
             'do' : 'Thursday',
             'fr' : 'Friday',
             'sa' : 'Saturday'}

  for filename in os.listdir(dir_region_volume):
    vol_path_list.append(dir_region_volume + filename)
    day = filename[-6:-4]
    vol_day_map[filename] = day_map[day]

  # store dataframes of a region, map with post object id
  region_vol_day_dict = {}
  for vol_path in vol_path_list:
    vol_df = pd.read_csv(vol_path, sep = ';')
    vol_df.rename(columns = {'BRIEFE' : 'LETTERS',
                             'PAKETE' : 'PACKAGES',
                             'SONSTIGE' : 'OTHERS',
                             'ROUTEPOS_ID' : 'RoutePosID'},
                  inplace = True)
    
    # combining files to a complete table for a region
    vol_po_df = pd.merge(po_file, vol_df, on = 'RoutePosID', how = 'left')
    vol_po_df = pd.merge(pp_file, vol_po_df, on='PostObjectId', how='right')
    vol_po_df = pd.merge(region_district_df, vol_po_df, on='PostObjectId', how='right')

    # store dataframes in dict
    day_key = vol_day_map[vol_path[-18:]]
    region_vol_day_dict[day_key] = vol_po_df
    
  return region_vol_day_dict

In [18]:
def generate_region_district(region_name):
  district_path = dir_region_instances = dir_loc + '/Instances/' + region_name + '/Districts'
  region_districts_list = []

  for filename in os.listdir(district_path):
    file_var = re.sub('.dat', '', filename)
    file_district = pd.read_csv(district_path + '/' + filename, sep='\t', skiprows = [0,1],
                                names=('PostObjectID', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5')
                                # usecols = [0]
                                )
    file_district['district'] = file_var
    region_districts_list.append(file_district)

  region_districts_df = pd.concat(region_districts_list, ignore_index = True)
  region_districts_df.drop(['dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5'], axis = 1, inplace = True)
  
  return region_districts_df

In [19]:
# generate instances

def generate_instances(region_vol_day, scenario_type, scenario_method, scenario_number, growth_factor):
  # df = region_vol_day.copy() #.copy() used to avoid recopying on the original dataframe
  df = region_vol_day

  sce_letters = 'scenario_' + str(scenario_number) + '_letter'
  sce_packages = 'scenario_' + str(scenario_number) + '_package'
  sce_others = 'scenario_' + str(scenario_number) + '_others'

  df[sce_letters] = df['LETTERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor ))
  df[sce_packages] = df['PACKAGES'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor))
  df[sce_others] = df['OTHERS'].apply(lambda x :scenario_type(pos_delivery = x, method = scenario_method, rate_pct = growth_factor))

  sce_all = 'scenario_' + str(scenario_number) + '_all'
  df[sce_all] = df[sce_letters] + df[sce_packages] + df[sce_others]

  return df

In [20]:
# generate real instances based on poisson

def random_poisson_instances(pos_delivery, method, rate_pct):
  rate = rate_pct/100
  rng = np.random.default_rng()
  poisson_dist = rng.poisson(lam = pos_delivery * (1 + rate), size = 52)

  if method == 'random':
    return random.choice(poisson_dist)
  if method == 'mode':
    return scipy.stats.mode(poisson_dist, keepdims = True)[0][0]
  else:
    return 'only options : [random, mode]'

In [21]:
# get start and end node for a district

def start_end_points(region_name, district, df_day):
    df_day_district = df_day[df_day['district'] == district]

    # get start and end node
    district_path = dir_loc + '/Instances/' + region_name + '/Districts/' + district + '.dat'
    rows_needed = [1]
    file_district = pd.read_csv(district_path, sep='\t', skiprows = lambda x : x not in rows_needed,
                                names=('dum_0', 'start_point', 'end_point', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5', 'dum_6')
                                )
    start_point = file_district['start_point'][0]
    end_point = file_district['end_point'][0]

    points = [start_point, end_point]

    return points

In [97]:
# generate original distance matrix and mapping for districts

def ori_generate_distance_matrix_map(region_name, save_loc):
      i = 1

      dir_region = save_loc + '/' + region_name + '_original'
      print(dir_region)
      if not os.path.exists(dir_region):
        os.mkdir(dir_region)
        print('dir_region doesnt exist')

      # dir_region_ori = dir_region + '/original'
      # print(dir_region_ori)
      # if not os.path.exists(dir_region):
      #   os.mkdir(dir_region)
      #   print('dir_ori doesnt exist')

      ori_dm_map_dict_region = {}
      # get distance file and dataframes
      distance_path = dir_loc + '/Instances/' + region_name + '/distances/'
      for filename in os.listdir(distance_path):
        ori_dm_map_dict_district = {}
        district_name = re.sub('.dat', '', filename)
        district_name = re.sub('distances_', '', district_name)

        district_distance = pd.read_csv(distance_path + filename,
                                    names=['pp_1', 'pp_2', 'dist']
                                    )
      
        distance_matrix_df = district_distance.pivot(index = 'pp_1', columns = 'pp_2', values = 'dist')

        # distance matrix
        distance_matrix_array = distance_matrix_df.to_numpy()
        distance_matrix_array = distance_matrix_array.tolist()
        ori_dm_map_dict_district['dm'] = distance_matrix_array
        pickle.dump(distance_matrix_array, open(dir_region + '/dm_' + "%s.p"%district_name, "wb"))

        # node mapping
        map_val = list(range(0, len(distance_matrix_df)))
        nodes = distance_matrix_df.index.values.tolist()
        mapping = dict(zip(map_val, nodes))
        ori_dm_map_dict_district['map'] = mapping
        pickle.dump(mapping, open(dir_region + '/map_' + "%s.p"%district_name, "wb"))

        ori_dm_map_dict_region[district_name] = ori_dm_map_dict_district
        print(i, ':', district_name, ':', len(distance_matrix_array))
        i = i+1

      return ori_dm_map_dict_region

In [61]:
# generate distance matrix and mapping for a scenario

def generate_distance_matrix_map(region_name, df_day_district, points, sce_col, district):
      scenario = sce_col

      col_use = ['PostPointId']
      col_use.append(scenario)
      df_day_district_scenario = df_day_district[col_use]
      
      # removing nodes with zero demand
      df_day_district_scenario_filtered = df_day_district_scenario[df_day_district_scenario[scenario] != 0]
      pp_id_day_district_scenario = df_day_district_scenario_filtered['PostPointId'].unique().tolist()
      
      # add start and end node if not in node list yet
      for point in points:
        if point not in pp_id_day_district_scenario:
          pp_id_day_district_scenario.append(point)

      # get distance file and dataframes
      distance_path = dir_loc + '/Instances/' + region_name + '/distances'
      district_distance = pd.read_csv(distance_path + '/distances_' + district + '.dat',
                                  names=['pp_1', 'pp_2', 'dist']
                                  )
      # remove unused postpoints
      district_distance_filtered = district_distance[(district_distance['pp_1'].isin(pp_id_day_district_scenario)) & (district_distance['pp_2'].isin(pp_id_day_district_scenario))]
      distance_matrix_df = district_distance_filtered.pivot(index = 'pp_1', columns = 'pp_2', values = 'dist')

      # distance matrix
      distance_matrix_array = distance_matrix_df.to_numpy()
      distance_matrix_array = distance_matrix_array.tolist()

      # node mapping
      map_val = list(range(0, len(distance_matrix_df)))
      nodes = distance_matrix_df.index.values.tolist()
      mapping = dict(zip(map_val, nodes))

      return distance_matrix_array, mapping

In [62]:
def create_dm_map(region, data, save_loc, rate):
  i = 1
  dir_region = save_loc + '/' + region
  if not os.path.exists(dir_region):
    os.mkdir(dir_region)


  # filter by day
  result_day = {}
  for day in data.keys():
    df_day = data[day]

    # filter by district
    result_district = {}
    for district in df_day['district'].value_counts().index.tolist():
      df_day_district = df_day[df_day['district'] == district]

      scenario_list = df_day_district.columns.tolist()[7:]

      # get start and end node
      points = start_end_points(region_name = region, district = district, df_day = df_day)
      
      # filter by scenario
      result_mail_demand = {}
      for scenario in scenario_list:
        # dir of mail
        mail_names = ['letter', 'package', 'others', 'all']
        for mail in mail_names:
          if mail in scenario:
            mail_current = mail

        # dir of demand
        demand_types = ['low', 'medium', 'high']
        for demand in demand_types:
          if demand in scenario:
            demand_current = demand

        # generate mapping and distance matrix
        distance_matrix, mapping = generate_distance_matrix_map(region_name = region, df_day_district = df_day_district,
                                                                points = points, sce_col = scenario, district = district)
        distance_matrix_name = 'dm_' + region + '_' + day + '_' + district + '_' + scenario + '_' + str(rate)
        mapping_name = 'map_' + region + '_' + day + '_' + district + '_' + scenario + '_' + str(rate)
        pickle.dump(distance_matrix, open(dir_region + '/' + "%s.p"%distance_matrix_name, "wb"))
        pickle.dump(mapping, open(dir_region + '/' + "%s.p"%mapping_name, "wb"))

        # log update
        print(i, ':', distance_matrix_name, ':', distance_matrix.shape)
        i = i+1

        result_end = {}
        result_end['dm'] = distance_matrix
        result_end['map'] = mapping
        
        mail_demand = mail_current + '_' + demand_current
        result_mail_demand[mail_demand] = result_end
      result_district[district] = result_mail_demand
    result_day[day] = result_district

  return result_day

In [9]:
# exact solution with concord
# no mapping yet

def concord_solver(distance_matrix):
  concorde = Concorde()
  concorde_sol = concorde.run_ProbATSP(distance_matrix)
  return concorde.route, concorde.optimal_value

In [90]:
# heuristic solution with LKH solver
# no mapping yet

class lkh:
    def __init__(self , dist_mat_pic_file):
        self.dist_mat_pic_file = dist_mat_pic_file
        self.dist_mat_file_name = dist_mat_pic_file.split(".")[0]
        print(self.dist_mat_file_name)

    def solve(self):
        '''
        solves the TSP instance using .exe file
        '''
        # Generate the .par file
        self.generate_parameter_file(self.dist_mat_file_name)

        # Generate the .atsp file
        dist_mat = self.read_pickle(self.dist_mat_pic_file)
        self.generate_atsp_file(dist_mat , self.dist_mat_file_name)

        try :    
            # Replace "E3k.0.par" with the actual parameter for your .exe file
            result = subprocess.check_output(["LKH-2.exe", f"{self.dist_mat_file_name}.par", "-S" , f"{self.dist_mat_file_name}_start.txt" ], stdin=subprocess.DEVNULL , timeout = 300)
            #result = subprocess.check_output(["LKH-2.exe", f"{self.dist_mat_file_name}.par", "-S" , f"{self.dist_mat_file_name}_start.txt" ], stdin=subprocess.DEVNULL , timeout = 300)
        
        except subprocess.TimeoutExpired:
            print("Execution timed out. Terminating the process.")
            # print(output)
        
        result_file_name = self.dist_mat_file_name + '.txt'
        print("Result is saved in : ",result_file_name)
        length , tour_section = self.get_obj_val_and_tour(result_file_name)
        length_rescaled = length/100

        print("Objective function value : ", length_rescaled)
        print("The routing : " , tour_section)

        return length_rescaled, tour_section

    
    def get_obj_val_and_tour(self , tour_file):
        """
        Extracts the length and tour section from a tour file.

        Args:
            tour_file (str): Path to the tour file.

        Returns:
            tuple: A tuple containing the length and tour section as a list.
        """
        length = None
        tour_section = []

        with open(tour_file, "r") as file:
            for line in file:
                line = line.strip()

                if line.startswith("COMMENT : Length"):
                    length = int(line.split("=")[1].strip())
                elif line == "TOUR_SECTION":
                    # Start reading the tour section
                    for tour_line in file:
                        tour_line = tour_line.strip()
                        if tour_line == "-1":
                            # End of tour section
                            break
                        tour_section.append(int(tour_line))

        return length, tour_section


    def get_obj_val(self , output):
        '''
        returns the objective value from the  long output value
        '''
        # Assuming `output` contains the captured output
        cost_min_match = re.search(r'Cost\.min = (\d+)', output)

        if cost_min_match:
            cost_min_value = int(cost_min_match.group(1))
            return cost_min_value
        else:
            print("Cost.min not found in the output.")
            return 0    
    
    def get_routing(self , filename):
        '''
        returns the routing in a numpy array
        '''
        start_keyword = 'TOUR_SECTION'
        end_keyword = '-1'
        with open(filename, 'r') as file:
            lines = []
            read_flag = False
            for line in file:
                if start_keyword in line:
                    read_flag = True
                    continue  # skip this line
                if read_flag:
                        if end_keyword in line:
                            read_flag = False
                            break
                        else:
                            lines.append(line.strip())

        lines = np.array(lines)
        return lines
    
    def generate_parameter_file(self , filename):
        runs = 1
        filename_atsp = filename + ".atsp"
        filename_txt = filename + ".txt"  
        filename_par = filename + ".par"

        with open(filename_par, 'w') as f:
            f.write(f"PROBLEM_FILE = {filename_atsp}\n")
            f.write(f"OUTPUT_TOUR_FILE = {filename_txt}\n")
            f.write(f"OPTIMUM = 40634081 \n")
            f.write(f"INITIAL_PERIOD = 1000 \n")
            f.write(f"MAX_CANDIDATES = 6 \n")
            f.write(f"MAX_TRIALS = 1000 \n")
            f.write(f"MOVE_TYPE = 6 \n")
            f.write(f"PATCHING_C = 6 \n")
            f.write(f"PATCHING_A = 5 \n")
            f.write(f"RECOMBINATION = GPX2 \n")
            f.write(f"RUNS = {runs} \n")

    
    def multiply_round_up(self , lst):
        '''
        input : list of lists having non integral values
        output : list of lists having integral values.
        Function to multiply, round up numbers,  and return as integers
        '''
        rounded_lst = np.rint(np.array(lst)*100).astype(int)
        rounded_lst = rounded_lst.tolist()
        return rounded_lst


    def read_pickle(self,pickle_path):
        # Open the pickle file in read-binary mode
        with open(pickle_path, 'rb') as f:
            # Load the object from the pickle file
            pic_obj = pickle.load(f)
        pic_obj = self.multiply_round_up(pic_obj)
        return pic_obj
    
    def generate_atsp_file(self , dist_mat, filename):   
        '''
        input : 
        a symmentric matrix  
        directory of a .tsp file you want to generate
        number of cities in the TSP

        Output : 
        a .tsp file generated in the filename directory
        '''
        # Defining the number of cities
        num_cities = len(dist_mat)

        # Write the coordinates to a .atsp file
        # fname = filename.split("/")[-1].split(".")[0]
        fname = filename 
        with open(filename+".atsp", 'w') as f:
            f.write(f"NAME: {fname}\n")
            # f.write("COMMENT: Randomly generated TSP instance\n")
            f.write("TYPE: TSP (M.~Hofmeister)\n")
            f.write(f"DIMENSION: {num_cities}\n") #becase the distance matrix doubles in size when converted to a symmentric instance
            f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
            f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
            # f.write("NODE_COORD_SECTION\n")
            f.write("DISPLAY_DATA_TYPE: NO_DISPLAY\n")
            f.write("EDGE_WEIGHT_SECTION\n")
            for i in range(len(dist_mat)):
                for j in range(len(dist_mat)):
                    f.write(f'{str(dist_mat[i][j])}\t')
                f.write("\n")
            f.write("EOF\n")

In [89]:
lst = [[0.456] , [0.1156], [6.3]]
# lst = np.array(np.array([0.456 , 0.1156, 6.3]))

rounded_lst = np.rint(np.array(lst)*100).astype(int)
rounded_lst = rounded_lst.tolist()
rounded_lst

[[46], [12], [630]]

#Process Examples (Using Functions)

In [26]:
%%time
# ['Warmsen', 'Uerze', 'Hannover 92']

region_vol_day_dict = generate_region_volume(region_name = "Warmsen")

CPU times: user 11.4 s, sys: 1.2 s, total: 12.6 s
Wall time: 28.5 s


In [ ]:
region_vol_day_dict['Monday'].head()

,PostObjectId,district,PostPointId,RoutePosID,LETTERS,PACKAGES,OTHERS
0,16,31606-14,16,AD14E08623FF9F5AD2293E7DEDB4F4B1,0.45,0.293333,0.0
1,17,31606-14,17,F48CB0CAB0BC117AE030007F0100574C,0.45,0.293333,0.0
2,18,31606-14,16,F48CB0CAB0C7117AE030007F0100574C,0.45,0.293333,0.0
3,19,31606-14,18,F48CB0CAB0C8117AE030007F0100574C,0.00,0.000000,0.0
4,20,31606-14,19,069AE286923C1800E040400A09131416,0.00,0.000000,0.0


In [27]:
%%time
#Creating the complete dictionary of volumes for a region
warmsen_instances = {}

#INPUT Rate of mail change 
# rate in percentage
rate = 50
rate_mapping = {'low' : -rate, 'medium' : 0, 'high' : rate}

# create instances for one region
for key in region_vol_day_dict.keys():
  # create instances per day
  for key_rate in rate_mapping:
    dummy = generate_instances(region_vol_day = region_vol_day_dict[key],
                               scenario_type = random_poisson_instances,
                               scenario_method = 'random',
                               scenario_number = key_rate,
                               growth_factor = rate_mapping[key_rate])
  warmsen_instances[key] = dummy

CPU times: user 15 s, sys: 234 ms, total: 15.2 s
Wall time: 15.5 s


In [ ]:
warmsen_instances['Tuesday'].head()

,PostObjectId,district,PostPointId,RoutePosID,LETTERS,PACKAGES,OTHERS,scenario_low_letter,scenario_low_package,scenario_low_others,scenario_low_all,scenario_medium_letter,scenario_medium_package,scenario_medium_others,scenario_medium_all,scenario_high_letter,scenario_high_package,scenario_high_others,scenario_high_all
0,16,31606-14,16,AD14E08623FF9F5AD2293E7DEDB4F4B1,5.796667,0.69,0.273333,3,0,0,3,5,0,0,5,10,1,0,11
1,17,31606-14,17,F48CB0CAB0BC117AE030007F0100574C,5.796667,0.69,0.273333,2,0,0,2,1,0,2,3,11,0,0,11
2,18,31606-14,16,F48CB0CAB0C7117AE030007F0100574C,5.796667,0.69,0.273333,3,1,2,6,5,0,0,5,5,1,0,6
3,19,31606-14,18,F48CB0CAB0C8117AE030007F0100574C,0.000000,0.00,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
4,20,31606-14,19,069AE286923C1800E040400A09131416,0.000000,0.00,0.000000,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(warmsen_instances['Tuesday']['scenario_low_all'].sum())
print(warmsen_instances['Tuesday']['scenario_medium_all'].sum())
print(warmsen_instances['Tuesday']['scenario_high_all'].sum())

4593
9279
13750


In [ ]:
z = warmsen_instances['Tuesday']


print(z[z['district'] == '31606-14']['scenario_low_all'].sum())
print(z[z['district'] == '31606-14']['scenario_medium_all'].sum())
print(z[z['district'] == '31606-14']['scenario_high_all'].sum())

262
566
859


In [71]:
%%time

# growth_rate_map_50 = {'high' : 'inc_50', 'low' : 'dec_50', 'medium' : 'base'}

result_region = create_dm_map(region = 'Warmsen', data = warmsen_instances,
                              save_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/',
                              rate = rate)

1 : dm_Warmsen_Tuesday_31603-06_scenario_low_letter_50 : (148, 148)
2 : dm_Warmsen_Tuesday_31603-06_scenario_low_package_50 : (42, 42)
3 : dm_Warmsen_Tuesday_31603-06_scenario_low_others_50 : (125, 125)
4 : dm_Warmsen_Tuesday_31603-06_scenario_low_all_50 : (240, 240)
5 : dm_Warmsen_Tuesday_31603-06_scenario_medium_letter_50 : (243, 243)
6 : dm_Warmsen_Tuesday_31603-06_scenario_medium_package_50 : (63, 63)
7 : dm_Warmsen_Tuesday_31603-06_scenario_medium_others_50 : (168, 168)
8 : dm_Warmsen_Tuesday_31603-06_scenario_medium_all_50 : (319, 319)
9 : dm_Warmsen_Tuesday_31603-06_scenario_high_letter_50 : (292, 292)
10 : dm_Warmsen_Tuesday_31603-06_scenario_high_package_50 : (107, 107)
11 : dm_Warmsen_Tuesday_31603-06_scenario_high_others_50 : (240, 240)
12 : dm_Warmsen_Tuesday_31603-06_scenario_high_all_50 : (369, 369)
13 : dm_Warmsen_Tuesday_31600-01_scenario_low_letter_50 : (149, 149)
14 : dm_Warmsen_Tuesday_31600-01_scenario_low_package_50 : (33, 33)
15 : dm_Warmsen_Tuesday_31600-01_scena

In [72]:
# result_region

name = 'warmsen_instances'

pickle.dump(result_region, open('/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results' + '/' + "%s.p"%name, "wb"))

In [73]:
warmze_instances= pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/warmsen_instances.p", "rb"))

In [74]:
warmze_instances.keys()

dict_keys(['Tuesday', 'Thursday', 'Friday', 'Wednesday', 'Monday', 'Saturday'])

In [75]:
warmze_instances['Saturday'].keys()

dict_keys(['31603-06', '31600-01', '31606-12', '31600-02', '31604-09', '31603-07', '31604-10', '31600-03', '31603-05', '31603-08', '31606-11', '31606-14', '31606-13', '31600-04'])

In [76]:
warmze_instances['Saturday']['31603-06'].keys()

dict_keys(['letter_low', 'package_low', 'others_low', 'all_low', 'letter_medium', 'package_medium', 'others_medium', 'all_medium', 'letter_high', 'package_high', 'others_high', 'all_high'])

In [77]:
warmze_instances['Saturday']['31603-06']['all_high'].keys()

dict_keys(['dm', 'map'])

In [78]:
warmze_instances['Saturday']['31603-06']['all_high']['dm']

array([[   0.  , 5879.32, 4906.21, ..., 6145.72, 6257.77, 6246.6 ],
       [5893.75,    0.  , 1388.68, ..., 1615.98, 1728.03, 1716.86],
       [4915.2 , 1356.05,    0.  , ..., 1733.57, 1845.62, 1834.45],
       ...,
       [6153.22, 1634.76, 1764.21, ...,    0.  ,  112.05,  123.22],
       [6265.27, 1746.81, 1876.26, ...,  112.05,    0.  ,  235.27],
       [6254.1 , 1735.64, 1865.09, ...,  123.22,  235.27,    0.  ]])

In [98]:
%%time

# original distance matrix

ori_region_dict = ori_generate_distance_matrix_map(region_name = 'Warmsen', save_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding')

/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Warmsen_original
dir_region doesnt exist
1 : 31606-14 : 311
2 : 31600-01 : 427
3 : 31606-13 : 306
4 : 31600-04 : 293
5 : 31600-02 : 417
6 : 31606-11 : 349
7 : 31603-08 : 360
8 : 31603-05 : 376
9 : 31603-06 : 424
10 : 31604-09 : 405
11 : 31604-10 : 380
12 : 31606-12 : 420
13 : 31600-03 : 372
14 : 31603-07 : 397
CPU times: user 1.22 s, sys: 102 ms, total: 1.32 s
Wall time: 1.65 s


In [99]:
ori_region_dict.keys()

dict_keys(['31606-14', '31600-01', '31606-13', '31600-04', '31600-02', '31606-11', '31603-08', '31603-05', '31603-06', '31604-09', '31604-10', '31606-12', '31600-03', '31603-07'])

In [100]:
ori_region_dict['31606-14'].keys()

dict_keys(['dm', 'map'])

In [104]:
# original region

name = 'warmsen_original'

pickle.dump(ori_region_dict, open('/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results' + '/' + "%s.p"%name, "wb"))

In [103]:
# ori_region_dict['31606-14']['map']

In [13]:
%%time

tes = [[1,2,3],[3,4,5],[9,10,11]]

concord_result_1, concord_result_2 = concord_solver(tes)

CPU times: user 2.5 ms, sys: 1.68 ms, total: 4.18 ms
Wall time: 4.2 ms


In [14]:
concord_result_1

[0, 1, 2]

In [15]:
concord_result_2

16.0

In [ ]:
%%time

s1 = lkh("dm_Warmsen_Saturday_31600-01_scenario_high_all.p")
length , tour_section = s1.solve()
